<a href="https://colab.research.google.com/github/nedokormysh/Stepik_AI_edu_intro_DL_NLP/blob/week_7_scraping/Fontanka_parsing_dirty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import time
import datetime
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
# from selenium import webdriver
from dataclasses import dataclass

In [ ]:
import requests

url = 'https://www.fontanka.ru/2023/11/01/news.html'
response = requests.get(url)
response

<Response [200]>

In [ ]:
tree = BeautifulSoup(response.content, 'html.parser')

In [ ]:
tree.find('a', {'class': 'IJdn'})

In [ ]:
# <button class="JTi9 JTez JTaip">Показать еще</button>

In [ ]:
# tree.find_all('section', {'class': 'Dp'})

In [ ]:
# tree.find_all('button')[22]

In [ ]:
# tree

In [ ]:
tree.find_all('button',  {'class': 'JTi9 JTez'})

[]

In [ ]:
tree.find('li', {'class': 'IJafj'})

<li class="IJafj"><div class="IJafl"><time class="IJgj"><i class="IJafn"></i><span>23:59</span></time><a class="IJhh IJafz" href="/doctorpiter/" title="Доктор Питер">Доктор Питер</a><div class="IJafp"></div></div><div class="IJafx"><div><a class="IJdn" href="https://doctorpiter.ru/zdorove/pediatr-burdakova-rasskazala-mozhno-li-pererasti-adenoidy-id919002/">Педиатр Бурдакова рассказала, можно ли «перерасти» аденоиды</a></div><a class="IJhh IJcx" href="/doctorpiter/" title="Доктор Питер">Доктор Питер</a></div></li>

In [ ]:
articles = tree.find_all('li', {'class': 'IJafj'})

In [ ]:
articles[0]

<li class="IJafj"><div class="IJafl"><time class="IJgj"><i class="IJafn"></i><span>23:59</span></time><a class="IJhh IJafz" href="/doctorpiter/" title="Доктор Питер">Доктор Питер</a><div class="IJafp"></div></div><div class="IJafx"><div><a class="IJdn" href="https://doctorpiter.ru/zdorove/pediatr-burdakova-rasskazala-mozhno-li-pererasti-adenoidy-id919002/">Педиатр Бурдакова рассказала, можно ли «перерасти» аденоиды</a></div><a class="IJhh IJcx" href="/doctorpiter/" title="Доктор Питер">Доктор Питер</a></div></li>

In [ ]:
x = articles[0].find_all('a', {'class': 'IJhh IJcx'})
x

[<a class="IJhh IJcx" href="/doctorpiter/" title="Доктор Питер">Доктор Питер</a>]

In [ ]:
x[0].get('title')

'Доктор Питер'

In [ ]:
x[0].text

'Доктор Питер'

In [ ]:
%%capture
!pip install selenium

In [ ]:
from selenium import webdriver

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('no-sandbox')
chrome_options.add_argument('disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
from selenium.webdriver.common.by import By

In [ ]:
driver.get('https://www.fontanka.ru/2023/11/01/news.html')
# click_button = driver.find_element(by=By.CLASS_NAME, value='JTi9 JTaip JTez')
# driver.find_elements_by_class_name('JTi9 JTaip JTez')

In [ ]:
# <button class="JTi9 JTez JTaip"> еще</button>

In [ ]:
# driver.find_element(By.NAME, 'ПОКАЗАТЬ ЕЩЕ')
# driver.find_elements(By.NAME, 'ПОКАЗАТЬ ЕЩЁ')

[]

In [ ]:
# https://www.youtube.com/watch?v=USrjHgO9Niw

In [ ]:
%%time

info = []

for article in tqdm(articles):
    topic = article.find_all('a', {'class': 'IJhh IJcx'})
    podclass = article.find_all('a', {'class': 'IJdn'})
    # title = podclass[0].text
    # print(title)
    # print(check)

    # if len(topic) == 0 or topic[0].get('href') == '/doctorpiter/':
    #    continue

    # scroll page to automatically load more articles
    # for i in tqdm(range(500), leave=False):
    #   try:
    #     driver.execute_script(f'window.scrollTo(0, document.body.scrollHeight - 1200)')
    #     time.sleep(1)
    #   except:
    #     pass

    # driver.execute_script("document.getElementsByClassName('JTi9 JTaip JTez')[0].click()")
    # driver.execute_script("document.find_element(by=By.CLASS_NAME, value='JTi9 JTaip JTez')[0].click()")

    # driver.get('https://www.fontanka.ru/2023/11/01/news.html')
    # click_button = driver.find_element(by=By.CLASS_NAME, value='JTi9 JTaip JTez')
    # click_button.click()

    # JTi9 JTez

    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    if len(topic) == 0:
       continue

    topic = topic[0].get('title')

    podclass = article.find_all('a', {'class': 'IJdn'})
    title = podclass[0].text

    url = 'https://www.fontanka.ru'+ podclass[0].get('href')

    if 'longreads' in url or 'doctorpiter' in url:
        continue

    response_inner = requests.get(url)
    tree = BeautifulSoup(response_inner.content, 'html.parser')


    check_add = tree.find('p', {'class': 'K-aon'}).text

    if check_add == 'Реклама':
       continue

    content_ = tree.find_all('div', {'class': 'CPah K5a3 K5ah'})

    content_txt = []
    [content_txt.append(el.text) for el in content_]



    content = ' '.join(content_txt)
    views = int(tree.find_all('span', {'class': 'BBhr primaryOverlineMobile'})[0].text)
    datetime = tree.find_all('span', {'itemprop': 'datePublished'})[0].text

    # print(title, topic)

    # url = 'https://www.fontanka.ru'+ podclass[0].get('href')
    row = {'url': url,
           'title': title,
           'content': content,
           'topic': topic,
           'views': views
        }

    info.append(row)

100%|██████████| 100/100 [01:39<00:00,  1.01it/s]

CPU times: user 22.5 s, sys: 314 ms, total: 22.9 s
Wall time: 1min 39s


In [ ]:
len(info)

77

In [ ]:
info[-1]

{'url': 'https://www.fontanka.ru/2023/11/01/72868709/',
 'title': 'RT: задержан бывший защитник сборной России по футболу Алексей Бугаев',
 'content': 'Бывший защитник сборной России по футболу Алексей Бугаев задержан. Об этом сообщает RT со ссылкой на источник 1 ноября. По данным телеканала, 42-летнего экс-футболиста остановили в Краснодаре в ночь на 29 октября. Предварительно, при личном осмотре у него нашли пакет с неизвестным порошком. Позже экспертиза подтвердила, что это 494 грамма наркотиков.Как пишут телеграм-каналы, бывший футболист заявил, что подрабатывал наркокурьером из-за трудного финансового положения в семье и делал закладки, когда его поймали правоохранители. Однако, как рассказал спортивному изданию «Чемпионат» отец бывшего футболиста, Бугаева задержали за неповиновение. По данным СМИ, бывший футболист арестован на семь суток и находится в спецприемнике.Алексей Бугаев играл за клубы «Локомотив», «Торпедо», «Томь», «Химки» и «Краснодар». За сборную России он провел сем

In [ ]:
url = 'https://www.fontanka.ru/2023/11/01/72871757/'

response = requests.get(url)
tree = BeautifulSoup(response.content, 'html.parser')

In [ ]:
content_ = tree.find_all('div', {'class': 'CPah K5a3 K5ah'})

content_txt = []
# for el in content_:
#    content_txt.append(el.text)
#   #  print(el.text)

[content_txt.append(el.text) for el in content_]

content = ' '.join(content_txt)
content

'В Москве у военкомата на улице Дмитрия Ульянова произошел взрыв, сообщает 1 ноября MSK1.RU. Журналистам говорят, что идут учения. «Хлопок был очень громкий, но видимых разрушений нет. Дома целы», — передает корреспондент.Ранее там обнаружили предмет, похожий на взрывное устройство. Территория оцеплена, на месте находятся полиция и МЧС.  «Вы сейчас сюда не пройдете. Тут ничего не происходит, просто учения. Идите в обход, пока сюда нельзя», — цитирует издание одного из силовиков. Однако водитель пожарной машины рассказал корреспонденту MSK1.RU, что «что-то нашли».'

In [ ]:
tree.find('p', {'class': 'K-aon'}).text

'Реклама'

In [ ]:
url = 'https://www.fontanka.ru/2023/11/01/72866645/'

response = requests.get(url)
tree = BeautifulSoup(response.content, 'html.parser')

(tree.find_all('span', {'itemprop': 'datePublished'})[0].text)

'1 ноября 2023, 23:27'

In [ ]:
dat = '1 ноября 2023, 23:27'
import datetime


In [ ]:
SLEEP = 2
DEPTH = 500
BASE_URL = 'https://www.fontanka.ru'
TOPICS = ['auto', 'world', 'politic', 'realty', 'society', 'sport', 'turizm', ]

In [ ]:
class my_spider(CrawlSpider):
    name = "fontanka"
    allowed_domains = ["fontanka.ru"]
    start_urls = ["http://www.fontanka.ru/fontanka/arc/news.html"]
    base_address = "http://www.fontanka.ru/"

    def parse(self, response):
        """
        Парсим стартовую страничку,
        находим ссылки на другие странички
        """
        for date in (date1, ....):
            # выбираем следующую дату
            site = self.base_address
            url = "%(site)s/%(date)s/news.html" % {
                "site": site,
                "date": date,
            }
            request = Request(url, callback = self.parse_page)
            yield request

    def parse_page(self, response):
        """
        Парсим каждую загруженную страничку.
        """
        list_site = sel.xpath('//a[contains(@class, pattern)]/@href')
        for i in list_site:
          #извлекаем содержимое i
            #записываем в базу данных тут или в pipelines
            # http://doc.scrapy.org/en/latest/topics/item-pipeline.html

# click button

In [ ]:
import re
import time
import datetime
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
# from selenium import webdriver
from dataclasses import dataclass

In [ ]:
%%capture
!pip install selenium

In [ ]:
## import dependencies
from selenium import webdriver
from selenium.webdriver.support.select import Select

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('no-sandbox')
chrome_options.add_argument('disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

## create an object of the chrome webdriver
# driver = webdriver.Chrome(executable_path = r'./chromedriver')
## open selenium URL in chrome browser


In [ ]:
from selenium.webdriver.common.by import By

In [ ]:
# <a href="/documentation/webdriver/" class="selenium-button selenium-webdriver text-uppercase fw-bold">Read more
# <i class="ps-3 fas fa-caret-right"></i></a>

In [ ]:
# <div class="selenium-button-container"><a href="/documentation/webdriver/" class="selenium-button selenium-webdriver text-uppercase fw-bold">Read more
# <i class="ps-3 fas fa-caret-right"></i></a></div>

In [ ]:
# driver.get('https://www.selenium.dev/')
# click_button = driver.find_elements(by=By.LINK_TEXT, value='Read more')
click_button = driver.find_element(by=By.CLASS_NAME , value='selenium-button-container')
click_button.click()

ElementClickInterceptedException: ignored

In [ ]:
driver.get('https://ria.ru/economy/')
# click_button = driver.find_elements(by=By.LINK_TEXT, value='Read more')
click_button = driver.find_element(by=By.CLASS_NAME , value='list-more')
print('Hello')
click_button

MaxRetryError: ignored

In [ ]:
driver.find_elements(By.CLASS_NAME, '')

In [ ]:
<button class="JTi9 JTez JTaip">Показать еще</button>

In [ ]:
driver.get('https://www.fontanka.ru/2023/11/01/news.html')

In [ ]:

# click_button =
driver.find_element(by=By.CLASS_NAME , value='JTi9 JTez JTaip')
# driver.find_elements_by_class_name('JTi9 JTaip JTez')

NoSuchElementException: ignored

In [ ]:
buttons = driver.find_elements(By.TAG_NAME, "button")
buttons

In [ ]:
all_btn = driver.find_elements(By.XPATH, "//button")

In [ ]:
buttons = driver.find_elements(By.CSS_SELECTOR, "button")

In [ ]:
# /html/body/div[1]/div/div[4]/div[2]/div/section/div/div/button
# <button class="JTi9 JTez JTaip">Показать еще</button>
# //*[@id="app"]/div/div[4]/div[2]/div/section/div/div/button